
# Assignment 4 for Course 1MS041
Make sure you pass the `# ... Test` cells and
 submit your solution notebook in the corresponding assignment on the course website. You can submit multiple times before the deadline and your highest score will be used.

---
## Assignment 4, PROBLEM 1
Maximum Points = 24


    This time the assignment only consists of one problem, but we will do a more comprehensive analysis instead.

Consider the dataset `Corona_NLP_train.csv` that you can get from the course website [git](https://github.com/datascience-intro/1MS041-2024/blob/main/notebooks/data/Corona_NLP_train.csv). The data is "Coronavirus tweets NLP - Text Classification" that can be found on [kaggle](https://www.kaggle.com/datasets/datatattle/covid-19-nlp-text-classification). The data has several columns, but we will only be working with `OriginalTweet`and `Sentiment`.

1. [3p] Load the data and filter out those tweets that have `Sentiment`=`Neutral`. Let $X$ represent the `OriginalTweet` and let 
    $$
        Y = 
        \begin{cases}
        1 & \text{if sentiment is towards positive}
        \\
        0 & \text{if sentiment is towards negative}.
        \end{cases}
    $$
    Put the resulting arrays into the variables $X$ and $Y$. Split the data into three parts, train/test/validation where train is 60% of the data, test is 15% and validation is 25% of the data. Do not do this randomly, this is to make sure that we all did the same splits (we are in this case assuming the data is IID as presented in the dataset). That is [train,test,validation] is the splitting layout.

2. [4p] There are many ways to solve this classification problem. The first main issue to resolve is to convert the $X$ variable to something that you can feed into a machine learning model. For instance, you can first use [`CountVectorizer`](https://scikit-learn.org/1.5/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) as the first step. The step that comes after should be a `LogisticRegression` model, but for this to work you need to put together the `CountVectorizer` and the `LogisticRegression` model into a [`Pipeline`](https://scikit-learn.org/1.5/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline). Fill in the variable `model` such that it accepts the raw text as input and outputs a number $0$ or $1$, make sure that `model.predict_proba` works for this. **Hint: You might need to play with the parameters of LogisticRegression to get convergence, make sure that it doesn't take too long or the autograder might kill your code**
3. [3p] Use your trained model and calculate the precision and recall on both classes. Fill in the corresponding variables with the answer.
4. [3p] Let us now define a cost function
    * A positive tweet that is classified as negative will have a cost of 1
    * A negative tweet that is classified as positive will have a cost of 5
    * Correct classifications cost 0
    
    complete filling the function `cost` to compute the cost of a prediction model under a certain prediction threshold (recall our precision recall lecture and the `predict_proba` function from trained models). 

5. [4p] Now, we wish to select the threshold of our classifier that minimizes the cost, fill in the selected threshold value in value `optimal_threshold`.
6. [4p] With your newly computed threshold value, compute the cost of putting this model in production by computing the cost using the validation data. Also provide a confidence interval of the cost using Hoeffdings inequality with a 99% confidence.
7. [3p] Let $t$ be the threshold you found and $f$ the model you fitted (one of the outputs of `predict_proba`), if we define the random variable
    $$
        C = (1-1_{f(X)\geq t})Y+5(1-Y)1_{f(X) \geq t}
    $$
    then $C$ denotes the cost of a randomly chosen tweet. In the previous step we estimated $\mathbb{E}[C]$ using the empirical mean. However, since the threshold is chosen to minimize cost it is likely that $C=0$ or $C=1$ than $C=5$ as such it will have a low variance. Compute the empirical variance of $C$ on the validation set. What would be the confidence interval if we used Bennett's inequality instead of Hoeffding in point 6 but with the computed empirical variance as our guess for the variance?

---

## 1. [3p] — Goal of Part 1

We are asked to:

1. Load the dataset `Corona_NLP_train.csv` using the fixed path  
   `../data/Corona_NLP_train.csv`.

2. Filter out rows where the column `Sentiment` is equal to `"Neutral"`.  
   Keep all other rows.

3. Create a binary response variable $Y$ defined by  

   $$
   Y =
   \begin{cases}
   1 & \text{if sentiment is towards positive},\\
   0 & \text{if sentiment is towards negative}.
   \end{cases}
   $$

   In the original dataset there are five sentiment levels:

   - Positive  
   - Extremely Positive  
   - Negative  
   - Extremely Negative  
   - Neutral  

   Here, "towards positive" means `Positive` or `Extremely Positive`,  
   and "towards negative" means `Negative` or `Extremely Negative`.

4. Let

   - $X$ be the array containing the text from the column `OriginalTweet`,  
   - $Y$ be the array containing the binary labels defined above.

   Both $X$ and $Y$ should be NumPy arrays of shape $(n_{\text{samples}},)$.

5. Split the filtered data **without randomness**, in the order that the
   observations appear in the CSV file, into three parts:

   - 60% $\rightarrow$ train  
   - 15% $\rightarrow$ test  
   - 25% $\rightarrow$ validation  

   If the total number of observations after removing neutrals is $N$, then

   $$
   N_{\text{train}} = \lfloor 0.6N \rfloor,
   $$
   $$
   N_{\text{test}} = \lfloor 0.15N \rfloor,
   $$
   $$
   N_{\text{valid}} = N - N_{\text{train}} - N_{\text{test}}.
   $$

   The corresponding splits for $X$ (and similarly for $Y$) are:

   - training set: $X[0 : N_{\text{train}}]$  
   - test set: $X[N_{\text{train}} : N_{\text{train}} + N_{\text{test}}]$  
   - validation set: the remaining part  
     $X[N_{\text{train}} + N_{\text{test}} : ]$.

---

### Step-by-step reasoning

1. Load the dataset with `pandas.read_csv` using the specified path.

2. Remove all rows where `Sentiment == "Neutral"` so that only
   clearly positive or negative tweets remain.

3. Extract the column `OriginalTweet` and store it as a NumPy array.
   This array will be $X$ and should have shape $(n_{\text{samples}},)$.

4. Convert the sentiment strings into numerical labels:

   - If the sentiment string contains `"Positive"` (either `Positive`
     or `Extremely Positive`), set the corresponding $Y$ value to $1$.
   - If the sentiment string contains `"Negative"` (either `Negative`
     or `Extremely Negative`), set the corresponding $Y$ value to $0$.

5. Compute $N_{\text{train}}$, $N_{\text{test}}$ and $N_{\text{valid}}$
   using the formulas above.

6. Use slicing on the arrays $X$ and $Y$ to obtain the three parts
   (train, test and validation) in the correct proportions and order.

7. Assign the resulting slices to the variables

   - `X_train`, `Y_train`  
   - `X_test`, `Y_test`  
   - `X_valid`, `Y_valid`

   keeping all of them as NumPy arrays.


In [11]:

# Part 1

# Load the data from the file specified in the problem definition and make sure that it is loaded using
# the search path `data/Corona_NLP_train.csv`. This is to make sure the autograder and your computer have the same
# file path and can load the data correctly.

# Contrary to how many other problems are structured, this problem actually requires you to
# have X on the shape (n_samples, ) that is a 1-dimensional array. Otherwise it will cause a bunch
# of errors in the autograder or also in for instance CountVectorizer.

# Make sure that all your data is numpy arrays and not pandas dataframes or series.
import pandas as pd
import numpy as np
# import os
# print(os.getcwd())
# print(os.listdir())

# Loading the file
# The dataset contains non-UTF8 characters (e.g. accented letters, emojis),
# which cause pandas to raise a UnicodeDecodeError when using default UTF-8.
# Therefore we specify encoding="latin1", which can read these characters safely.
data = pd.read_csv("data/Corona_NLP_train.csv", encoding="latin1")



# Remove Neutral sentiments
data = data[data["Sentiment"] != "Neutral"]

# Extract X as numpy array of strings
X = data["OriginalTweet"].values

# Convert Sentiment → binary Y
Y = np.where(
    data["Sentiment"].str.contains("Positive"), 
    1, 
    0
)

# Determine split sizes
N = len(X)
N_train = int(0.60 * N)
N_test = int(0.15 * N)
N_valid = N - N_train - N_test

# Deterministic split
X_train = X[:N_train]
Y_train = Y[:N_train]

X_test = X[N_train:N_train + N_test]
Y_test = Y[N_train:N_train + N_test]

X_valid = X[N_train + N_test:]
Y_valid = Y[N_train + N_test:]


---

## 2. [ ?p ] — Training a Sentiment Classification Model

After removing all tweets with `Sentiment = "Neutral"`, we now want to train a
machine learning model that can predict the binary sentiment variable

$$
Y =
\begin{cases}
1 & \text{if the sentiment is towards positive},\\[6pt]
0 & \text{if the sentiment is towards negative}.
\end{cases}
$$

The input to the model is the array $X$, which contains **raw tweet strings**.
Since machine learning algorithms cannot operate directly on text, we first need
to convert each tweet into a numerical representation.

### **Vectorization of Text**
We use a **TF–IDF vectorizer**, which transforms each tweet into a vector that
measures how important each word is compared to the full dataset.  
Mathematically, TF–IDF assigns each token $t$ in document $d$ a weight:

$$
\text{tfidf}(t,d) = \text{tf}(t,d) \cdot \log\left( \frac{N}{\text{df}(t)} \right),
$$

where  
- $\text{tf}(t,d)$ is the term frequency of token $t$ in tweet $d$,  
- $\text{df}(t)$ is the number of tweets containing $t$,  
- $N$ is the total number of tweets.

This converts $X$ into a numerical matrix suitable for machine learning.

### **Classifier**
We use **Logistic Regression**, a standard model for binary outcomes:

$$
P(Y=1 \mid x) = \frac{1}{1 + e^{-(\beta_0 + \beta^\top x)}}.
$$

The classifier learns coefficients $\beta$ that best separate positive from
negative tweets.

### **Pipeline**
Because the autograder will call the model directly on raw strings (not
preprocessed text), we must combine both steps:

1. TF–IDF vectorizer  
2. Logistic Regression classifier  

into a single object:

$$
\text{model} = \text{Vectorizer} \circ \text{Classifier}.
$$

This ensures that calling

```python
model.predict(X_test)


In [12]:

# Part 2

# Train a machine learning model or pipeline that can take the raw strings from X and predict Y=0,1 depending on the
# sentiment of the tweet. Store the trained model in the variable `model`.

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

# We create a scikit-learn Pipeline so that:
# - The vectorizer transforms raw text into numeric features.
# - The classifier learns to predict Y = 0 or 1.
# Using a pipeline ensures that `model.predict(X)` works directly on raw tweets,
# which is required by the autograder (it will not provide pre-vectorized input).

model = Pipeline([
    ("tfidf", TfidfVectorizer()),         # Step 1: convert text → TF-IDF matrix
    ("logreg", LogisticRegression(
        max_iter=1000                     # Increase iterations to ensure convergence
    ))                                    # Step 2: logistic regression classifier
])

# Train the model using the training split
model.fit(X_train, Y_train)


,steps,"[('tfidf', ...), ('logreg', ...)]"
,transform_input,None
,memory,None
,verbose,False
,input,'content'
,encoding,'utf-8'
,decode_error,'strict'
,strip_accents,None
,lowercase,True
,preprocessor,None
,tokenizer,None


3. [3p] Use your trained model and calculate the precision and recall on both classes. Fill in the corresponding variables with the answer.

## 3. [3p] — Precision and Recall for Both Classes

Once the model has been trained, we evaluate its performance on the **test set**
using two important classification metrics: **precision** and **recall**.

The model predicts a binary label  
$\hat{Y} \in \{0,1\}$   
based on the raw input text in `X_test`.

### **Precision**

Precision measures how many predicted positives (or negatives) were actually correct.

For class 1 (positive sentiment):

$$
\text{Precision}_1 =
\frac{\text{True Positives}}{\text{True Positives} + \text{False Positives}}.
$$

For class 0 (negative sentiment):

$$
\text{Precision}_0 =
\frac{\text{True Negatives}}{\text{True Negatives} + \text{False Negatives}}.
$$

### **Recall**

Recall measures how many actual positives (or negatives) we correctly found.

For class 1:

$$
\text{Recall}_1 =
\frac{\text{True Positives}}{\text{True Positives} + \text{False Negatives}}.
$$

For class 0:

$$
\text{Recall}_0 =
\frac{\text{True Negatives}}{\text{True Negatives} + \text{False Positives}}.
$$

### **How we compute the metrics**

We:

1. Use the trained model to generate predictions  
   $$\hat{Y} = \text{model.predict}(X_{\text{test}}).$$

2. Call `precision_score` and `recall_score` from scikit-learn.

3. We compute the scores **separately for each class** by specifying  
   `pos_label=1` or `pos_label=0`.

4. Store the resulting values in the variables:

- `precision_0`, `precision_1`  
- `recall_0`, `recall_1`

as required by the assignment.

In [13]:

# Part 3

# Evaluate the model on the test set and calculate precision, and recall on both classes. Store the results in the
# variables `precision_0`, `precision_1`, `recall_0`, `recall_1`.

from sklearn.metrics import precision_score, recall_score

# Predict labels on the test set
Y_pred = model.predict(X_test)

# Precision for class 0 (negative sentiment)
# pos_label=0 means we treat class 0 as the "positive" class in the formula,
# making precision_0 = TN / (TN + FN)
precision_0 = precision_score(Y_test, Y_pred, pos_label=0)

# Precision for class 1 (positive sentiment)
precision_1 = precision_score(Y_test, Y_pred, pos_label=1)

# Recall for class 0 (negative sentiment)
# recall_0 = TN / (TN + FP)
recall_0 = recall_score(Y_test, Y_pred, pos_label=0)

# Recall for class 1
recall_1 = recall_score(Y_test, Y_pred, pos_label=1)


4. [3p] Let us now define a cost function
    * A positive tweet that is classified as negative will have a cost of 1
    * A negative tweet that is classified as positive will have a cost of 5
    * Correct classifications cost 0
    
    complete filling the function `cost` to compute the cost of a prediction model under a certain prediction threshold (recall our precision recall lecture and the `predict_proba` function from trained models). 

---

## 4. [3p] — Defining a Cost Function Using Thresholded Predictions

In this part we introduce a cost function that assigns different penalties
depending on the type of misclassification the model makes.  
We let the response variable take values

$$
Y \in \{0,1\},
$$

where

- $Y = 1$ means **positive sentiment**,  
- $Y = 0$ means **negative sentiment**.

### Probability-based decision rule

The trained model can estimate the probability that a tweet has positive
sentiment:

$$
p(x) = P(Y = 1 \mid x).
$$

In scikit-learn this is obtained using

model.predict_proba(X)



This returns two columns:

- column 0: $( P(Y = 0 \mid x))$
- column 1: $( P(Y = 1 \mid x))$

## Decision rule using a threshold

We classify a tweet as **positive** if

$$p(x) \ge \text{threshold},$$

and classify it as **negative** otherwise.

This generalizes the Bayes classifier from the lecture, where the threshold was \( 0.5 \) when misclassification costs were equal.

---

## Cost structure

We now assign different penalties:

1. A positive tweet that is classified as negative will have a cost of **1**.  
2. A negative tweet that is classified as positive will have a cost of **5**.  
3. Correct classifications cost **0**.

Formally, for each observation,

$$
\text{cost}(Y, \hat{Y}) =
\begin{cases}
1 & \text{if } Y = 1 \text{ and } \hat{Y} = 0, \\
5 & \text{if } Y = 0 \text{ and } \hat{Y} = 1, \\
0 & \text{otherwise}.
\end{cases}
$$

---

The function `cost(model, threshold, X, Y)` should:

1. Use `model.predict_proba(X)` to get the probabilities $( p(x) = P(Y = 1 \mid x) )$ .

2. Convert these probabilities into predictions using the given threshold  
   (predict 1 if $( p(x) \ge \text{threshold} )$, otherwise 0).

3. Compute the costs:
   - cost **1** for false negatives (true $(Y = 1)$, predicted 0),
   - cost **5** for false positives (true $(Y = 0)$, predicted 1),
   - cost **0** otherwise.

4. Compute the **average cost** over all observations and return it.

The average cost is

$$
\text{AverageCost} = \frac{1}{N} \sum_{i=1}^N \text{cost}(Y_i, \hat{Y}_i).
$$



In [ ]:

# Part 4

def cost(model,threshold,X,Y):
    # Hint, make sure that the model has a predict_proba method
    # think about how the decision is made based on the probabilities
    # and how the threshold can be used to make the decision.
    # For reference take a look at the lecture notes "Bayes classifier"
    # which contains how the decision is made based on the probabilities when the threshold is 0.5.
    
    # Fill in what is missing to compute the cost and return it
    # Note that we are interested in average cost
    


    # We need class probabilities to make threshold-based decisions.
    # predict_proba returns an array of shape (n_samples, 2)
    # where column 1 is P(Y=1 | x).
    probs = model.predict_proba(X)[:, 1]  # probability of class 1 (positive)
    
    # Apply the decision rule:
    # predict positive (1) if prob >= threshold, otherwise negative (0)
    Y_pred = (probs >= threshold).astype(int)
    
    # Compute costs:
    # False negative (Y=1, Y_pred=0) --> cost 1
    fn_cost = (Y == 1) & (Y_pred == 0)
    
    # False positive (Y=0, Y_pred=1) --> cost 5
    fp_cost = (Y == 0) & (Y_pred == 1)
    
    # Total cost per observation
    costs = 1 * fn_cost + 5 * fp_cost
    
    # Return average cost
    return costs.mean()

    

5. [4p] Now, we wish to select the threshold of our classifier that minimizes the cost, fill in the selected threshold value in value `optimal_threshold`.

## 5. [4p]

In this part we want to choose the threshold of our classifier that **minimizes the expected cost** on the test set.  
Our logistic regression model produces estimated posterior probabilities

$$
p(x) = P(Y = 1 \mid x),
$$

and we classify an observation as belonging to class 1 when  
$( p(x) \ge t )$, where $( t )$ is the decision threshold.

---

### Step 1: Understanding the Cost Function

We are given the following misclassification costs:

- False negative (predict 0 when $(Y=1)$):
  $$
  c_{\text{FN}} = 1
  $$

- False positive (predict 1 when $(Y=0)$):
  $$
  c_{\text{FP}} = 5
  $$

Thus, the classifier must be more cautious before predicting class 1, because false positives are much more expensive.

---

### Step 2: Conditional Risk for Each Decision

For a fixed observation $(x)$, the model estimates

$$
p(x) = P(Y=1 \mid x), \qquad 1 - p(x) = P(Y=0 \mid x).
$$

If we **predict class 1**, the conditional expected cost is

$$
R(1 \mid x) = c_{\text{FP}} \cdot P(Y=0 \mid x)
= 5 (1 - p(x)).
$$

If we **predict class 0**, the conditional expected cost is

$$
R(0 \mid x) = c_{\text{FN}} \cdot P(Y=1 \mid x)
= 1 \cdot p(x).
$$

---

### Step 3: Deriving the Optimal Theoretical Threshold

The Bayes classifier chooses the class with the smaller conditional risk:

$$
\text{Predict 1 if } R(1 \mid x) < R(0 \mid x).
$$

Insert the formulas:

$$
5(1 - p(x)) < p(x).
$$

Solve for \( p(x) \):

$$
5 - 5p(x) < p(x)
$$

$$
5 < 6p(x)
$$

$$
p(x) > \frac{5}{6}.
$$

Thus, the **Bayes-optimal threshold** is

$$
t^* = \frac{5}{6} \approx 0.8333.
$$

This means that, to minimize expected cost, we should only classify a tweet as positive when the models probability exceeds approximately $(0.83)$.

---

### Step 4: Empirical Optimization on the Test Set

Although the theoretical value is $(t^* = 5/6)$, in practice we estimate the optimal threshold by **minimizing the empirical average cost** on the test set.

For a given threshold $(t)$:

- Predict 1 if  
  $$
  p(x) \ge t,
  $$
- Predict 0 otherwise.

The cost for each observation is

- $(1)$ if it is a false negative,
- $(5)$ if it is a false positive,
- $(0)$ if correctly classified,

and the **average cost** is computed over all test samples.

---

### Step 5: Search Over Threshold Values

To find the threshold with minimal test-set cost, we evaluate the empirical average cost for a sequence of thresholds

$$
t \in \{0.0, 0.001, 0.002, \ldots, 1.0\}.
$$

For each $(t)$ we compute

$$
\widehat{C}(t)
$$

and choose the threshold that yields

$$
\min_{t \in [0,1]} \ \widehat{C}(t).
$$

This empirical minimizer is stored in

- `optimal_threshold`
- `cost_at_optimal_threshold`

---

### Summary

- Theoretical decision rule:

  $$
  t^* = \frac{5}{6}.
  $$

- Practical decision rule:

  Evaluate the cost on the test set for many thresholds  
  and choose the one that gives the smallest average cost.

This ensures that the classifier is tuned specifically to minimize the given misclassification costs on the real data.


In [17]:

# Part 5

# Find the optimal threshold for the model on the test set. Store the threshold in the variable `optimal_threshold`
# and the cost at the optimal threshold in the variable `cost_at_optimal_threshold` evaluated on the test set.



# Create a grid of thresholds between 0 and 1
thresholds = np.linspace(0.0, 1.0, 1001)  # step size 0.001

# Compute the cost for each threshold on the test set
costs = np.array([cost(model, t, X_test, Y_test) for t in thresholds])

# Find the index of the minimum cost
best_idx = np.argmin(costs)

# Extract the optimal threshold and the corresponding cost
optimal_threshold = thresholds[best_idx]
cost_at_optimal_threshold = costs[best_idx]

print("Optimal Threshold: ", optimal_threshold)
print("Cost at optimal threshold: ", cost_at_optimal_threshold)
#optimal_threshold, cost_at_optimal_threshold


Optimal Threshold:  0.677
Cost at optimal threshold:  0.3066188197767145


6. [4p] With your newly computed threshold value, compute the cost of putting this model in production by computing the cost using the validation data. Also provide a confidence interval of the cost using Hoeffdings inequality with a 99% confidence.


## 6. [4p]

In this part we evaluate how well our chosen threshold would perform **when the model is deployed in production**.  
To simulate this, we use the **validation dataset**, which was not involved in training or threshold selection.

We compute two things:

1. The **empirical cost** on the validation set using the optimal threshold obtained in Part 5.
2. A **99% confidence interval** for this cost using **Hoeffding's inequality**.

---

### Step 1: Cost on the Validation Set

Given the optimal threshold $t^*$ found earlier, we compute

$$
\widehat{C}_{\text{valid}} = \frac{1}{n} \sum_{i=1}^n Z_i,
$$

where $Z_i$ is the cost incurred for the $i$th validation observation:

- $Z_i = 1$ for a false negative  
- $Z_i = 5$ for a false positive  
- $Z_i = 0$ for a correct prediction

The function $cost(model, t^*, X_{\text{valid}}, Y_{\text{valid}})$ returns exactly this empirical mean.

---

### Step 2: Confidence Interval Using Hoeffding's Inequality

Let $\widehat{C}$ be the empirical cost and assume that each cost value $Z_i$ is bounded:

$$
0 \le Z_i \le 5.
$$

Hoeffding’s inequality states that with probability at least $1 - \delta$,

$$
\left| \widehat{C} - C \right| \le (b - a) \sqrt{\frac{\ln(2/\delta)}{2n}},
$$

where:

- $a = 0$ (minimum cost per observation)  
- $b = 5$ (maximum cost per observation)  
- $n$ is the validation set size  
- $\delta = 0.01$ for a 99% confidence interval

Thus the radius of the interval is

$$
\epsilon = 5 \sqrt{\frac{\ln(2 / 0.01)}{2n}}.
$$

The 99% confidence interval for the true expected cost $C$ is

$$
\left( \widehat{C} - \epsilon,\; \widehat{C} + \epsilon \right).
$$

This gives an interval guaranteed by Hoeffding's inequality to contain the true cost with confidence at least 99%.

---

### Summary

- Compute empirical cost on the validation set using $t^*$:

  $$
  \widehat{C}_{\text{valid}} = \text{cost}(model, t^*, X_{\text{valid}}, Y_{\text{valid}}).
  $$

- Compute Hoeffding radius:

  $$
  \epsilon = 5 \sqrt{\frac{\ln(200)}{2n}}.
  $$

- Report the confidence interval:

  $$
  (\widehat{C}_{\text{valid}} - \epsilon,\; \widehat{C}_{\text{valid}} + \epsilon).
  $$

This provides a statistically justified estimate of how the model is expected to perform when deployed in practice.


In [20]:

# Part 6

# Compute cost on the validation set using the optimal threshold
cost_at_optimal_threshold_valid = cost(model, optimal_threshold, X_valid, Y_valid)

# Hoeffding's inequality for 99% confidence interval
n = len(Y_valid)
a, b = 0, 5  # minimum and maximum possible cost per observation
delta = 0.01

# Compute epsilon (Hoeffding bound)
epsilon = (b - a) * np.sqrt(np.log(2 / delta) / (2 * n))

# Build the confidence interval
cost_interval_valid = (cost_at_optimal_threshold_valid - epsilon, cost_at_optimal_threshold_valid + epsilon)

print("Cost at optimal threshold valid: ", cost_at_optimal_threshold_valid)
print("cost interval valid: ", cost_interval_valid)

# Required by the autograder
assert type(cost_interval_valid) == tuple
assert len(cost_interval_valid) == 2




Cost at optimal threshold valid:  0.32241090648170295
cost interval valid:  (np.float64(0.23341522986171315), np.float64(0.41140658310169276))


7. [3p] Let $t$ be the threshold you found and $f$ the model you fitted (one of the outputs of `predict_proba`), if we define the random variable
    $$
        C = (1-1_{f(X)\geq t})Y+5(1-Y)1_{f(X) \geq t}
    $$
    then $C$ denotes the cost of a randomly chosen tweet. In the previous step we estimated $\mathbb{E}[C]$ using the empirical mean. However, since the threshold is chosen to minimize cost it is likely that $C=0$ or $C=1$ than $C=5$ as such it will have a low variance. Compute the empirical variance of $C$ on the validation set. What would be the confidence interval if we used Bennett's inequality instead of Hoeffding in point 6 but with the computed empirical variance as our guess for the variance?

## 7. [3p]

Let $t$ be the threshold found in the previous step and $f$ the fitted model
(one of the outputs of `predict_proba`). Define the random variable

$$
C = (1 - 1_{\{ f(X) \ge t \}})Y + 5(1 - Y)\, 1_{\{ f(X) \ge t \}}
$$

which denotes the cost of a randomly chosen tweet.

### Distribution of $C$

Let $I = 1_{\{ f(X) \ge t \}}$.

We consider the different cases depending on the true label $Y \in \{0,1\}$.

- If $Y = 1$ (positive tweet):

  $$
  C = (1 - I)\cdot 1 + 5(1-1)\cdot I = 1 - I
  $$

  - If $I = 1$ then $C = 0$ (true positive).  
  - If $I = 0$ then $C = 1$ (false negative).

- If $Y = 0$ (negative tweet):

  $$
  C = (1 - I)\cdot 0 + 5(1-0)\cdot I = 5I
  $$

  - If $I = 1$ then $C = 5$ (false positive).  
  - If $I = 0$ then $C = 0$ (true negative).

Thus

$$
C \in \{0,1,5\}
$$

and

$$
0 \le C \le 5.
$$

### Empirical mean and variance of $C$

On the validation set $(X_{\text{valid}}, Y_{\text{valid}})$ compute

$$
p_i = f(X_i), \qquad
I_i = 1_{\{ p_i \ge t \}}, \qquad
C_i = (1 - I_i)\, Y_i + 5(1 - Y_i)\, I_i
$$

for $i = 1, \dots, n$ where $n = |X_{\text{valid}}|$.

The empirical mean is

$$
\overline{C} = \frac{1}{n}\sum_{i=1}^n C_i
$$

which equals `cost_at_optimal_threshold_valid`.

The empirical variance is

$$
\widehat{\sigma}^2 = \frac{1}{n}\sum_{i=1}^n (C_i - \overline{C})^2.
$$

This is stored as `variance_of_C`.

### Bennett's inequality and confidence interval

Since $0 \le C_i \le b$ with $b = 5$, Bennett’s inequality gives

$$
\mathbb{P}\left(
\left|\overline{C} - \mathbb{E}[C]\right| \ge \varepsilon
\right)
\le
2 \exp\left(
-\frac{n\widehat{\sigma}^2}{b^2}\,
h\!\left( \frac{b\varepsilon}{\widehat{\sigma}^2} \right)
\right)
$$

where

$$
h(u) = (1+u)\log(1+u) - u.
$$

For a $99\%$ confidence interval, set $\delta = 0.01$ and solve for $\varepsilon$ in

$$
2\exp\left(
-\frac{n\widehat{\sigma}^2}{b^2}\,
h\!\left( \frac{b\varepsilon}{\widehat{\sigma}^2} \right)
\right)
= \delta.
$$

Taking logs gives

$$
h\!\left( \frac{b\varepsilon}{\widehat{\sigma}^2} \right)
=
\frac{b^2}{n \widehat{\sigma}^2}
\log\left(\frac{2}{\delta}\right).
$$

We solve this equation numerically to obtain $\varepsilon_B$.

Thus the Bennett 99% confidence interval is

$$
\left[
\overline{C} - \varepsilon_B,\;
\overline{C} + \varepsilon_B
\right]
$$

i.e.

$$
\left[
\texttt{cost\_at\_optimal\_threshold\_valid} - \varepsilon_B,\;
\texttt{cost\_at\_optimal\_threshold\_valid} + \varepsilon_B
\right].
$$


In [21]:

# Part 7
import numpy as np

# Compute C on the validation set
probs_valid = model.predict_proba(X_valid)[:, 1]
I_valid = (probs_valid >= optimal_threshold).astype(int)

# C = (1 - 1_{f(X)>=t}) * Y + 5 * (1 - Y) * 1_{f(X)>=t}
C_valid = (1 - I_valid) * Y_valid + 5 * (1 - Y_valid) * I_valid

# Empirical variance of C (population variance, denominator n)
variance_of_C = C_valid.var()

# Bennett's inequality-based confidence interval (99%)
n = len(C_valid)
b = 5.0
delta = 0.01

def h(u):
    return (1.0 + u) * np.log(1.0 + u) - u

def bennett_epsilon(n, var_hat, b, delta):
    """
    Solve for epsilon in the two-sided Bennett inequality:
        2 * exp( - n * var_hat / b^2 * h(b * epsilon / var_hat) ) = delta
    using a simple binary search over epsilon in [0, b].
    """
    if var_hat == 0.0:
        # All costs are identical, no deviation
        return 0.0

    target = (b ** 2) * np.log(2.0 / delta) / (n * var_hat)

    lo, hi = 0.0, b
    for _ in range(100):
        mid = 0.5 * (lo + hi)
        u = b * mid / var_hat
        val = h(u)
        if val < target:
            lo = mid
        else:
            hi = mid
    return hi

epsilon_B = bennett_epsilon(n, variance_of_C, b, delta)

interval_of_C = (
    cost_at_optimal_threshold_valid - epsilon_B,
    cost_at_optimal_threshold_valid + epsilon_B
)

print("Empirical variance of C on validation set:", variance_of_C)
print("Bennett 99% confidence interval for E[C]:", interval_of_C)

assert(type(interval_of_C) == tuple)
assert(len(interval_of_C) == 2)


assert(type(interval_of_C) == tuple)
assert(len(interval_of_C) == 2)

Empirical variance of C on validation set: 0.7303013867645695
Bennett 99% confidence interval for E[C]: (np.float64(0.2909508033233758), np.float64(0.3538710096400301))
